In [57]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import parse_kraken_orderbook
from hydradx.model.amm.arbitrage_agent import get_arb_swaps, calculate_profit, execute_arb
from hydradx.model.amm.centralized_market import CentralizedMarket, OrderBook
from hydradxapi import HydraDX
from pprint import pprint
import requests



import json, os


In [58]:
tkn_pairs = ['HDXUSD', 'DOTUSD', 'XETHZUSD', 'DOTETH', 'XXBTZUSD', 'DOTXBT', 'XETHXXBT', 'ASTRUSD', 'CFGUSD', 'BNCUSD', 'GLMRUSD']
op_tkn_pairs = [('HDX','USDT'), ('DOT','USDT'), ('WETH','USDT'), ('DOT','WETH'), ('WBTC','USDT'), ('DOT','WBTC'), ('WETH','WBTC'), ('ASTR','USDT'), ('CFG','USDT'), ('BNC','USDT'), ('GLMR','USDT')]
order_books = {}
for tkn_pair in tkn_pairs:
    resp = requests.get('https://api.kraken.com/0/public/Depth?pair=' + tkn_pair)
    y = resp.json()
    order_books[tkn_pair] = y['result'][tkn_pair]
    

# f = open('data/hdxusd_orderbook.json', 'r')
# y = json.loads(f.read())
# f = open('data/dotusd_orderbook.json', 'r')
# z = json.loads(f.read())
# hdxusd_orderbook = y['result']['HDXUSD']
# dotusd_orderbook = z['result']['DOTUSD']

# pprint(hdxusd_orderbook)



In [59]:
RPC="wss://hydradx-rpc.dwellir.com"

chain = HydraDX(RPC) 
chain.connect()

# Omnipool metadata
metadata = [
    {'symbol': 'HDX', 'id': 0, 'decimals': 12},
    {'symbol': 'LRNA', 'id': 1, 'decimals': 12},
    {'symbol': 'DAI', 'id': 2, 'decimals': 18},
    {'symbol': 'WBTC', 'id': 3, 'decimals': 8},
    {'symbol': 'WETH', 'id': 4, 'decimals': 18},
    {'symbol': 'DOT', 'id': 5, 'decimals': 10},
    {'symbol': 'APE', 'id': 6, 'decimals': 18},
    {'symbol': 'USDC', 'id': 7, 'decimals': 6},
    {'symbol': 'PHA', 'id': 8, 'decimals': 12},
    {'symbol': 'ASTR', 'id': 9, 'decimals': 18},
    {'symbol': 'USDT', 'id': 10, 'decimals': 6},
    {'symbol': 'IBTC', 'id': 11, 'decimals': 8},
    {'symbol': 'ZTG', 'id': 12, 'decimals': 10},
    {'symbol': 'CFG', 'id': 13, 'decimals': 18},
    {'symbol': 'BNC', 'id': 14, 'decimals': 12},
    {'symbol': 'vDOT', 'id': 15, 'decimals': 10},
    {'symbol': 'GLMR', 'id': 16, 'decimals': 18},
    {'symbol': 'INTR', 'id': 17, 'decimals': 10},
    {'symbol': 'DAI', 'id': 18, 'decimals': 18},
    {'symbol': 'WBTC', 'id': 19, 'decimals': 8},
    {'symbol': 'WETH', 'id': 20, 'decimals': 18},
    {'symbol': 'USDC', 'id': 21, 'decimals': 6},
    {'symbol': 'USDC', 'id': 22, 'decimals': 6},
    {'symbol': 'USDT', 'id': 23, 'decimals': 6},
    ]

#Asset: 0
#	Reserve: 0
#	LRNA: 20220634396578953
#	Shares: 40600077930826935893
#	Protocol: 3694884494247885459

tokens = {}

asset_list = ['USDT','DOT','HDX','WETH','WBTC','IBTC','ASTR','CFG','BNC','GLMR','ZTG']
# id_list = [i for i in range(len(metadata)) if metadata[i]['symbol'] in asset_list]
id_list = [0,5,10,4,3,11,9,13,14,16,12]

for i in id_list:
    try:
        state = chain.api.omnipool.asset_state(i)
    except:
        continue
    tokens[metadata[i]['symbol']] = {
        'liquidity': state.reserve / 10**metadata[i]['decimals'],
        'LRNA': state.hub_reserve / 10**12
    }
    
# fees = {{'asset_fee': chain.api.fees.asset_fees(i)} for i in id_list}
fees = {}
for i in id_list:
    fee = chain.api.fees.asset_fees(i)
    fees[metadata[i]['symbol']] = {"asset_fee": fee.asset_fee / 100, "protocol_fee": fee.protocol_fee / 100}
    
chain.close()
pprint(fees)

# get Omnipool data
# tokens = {
#     'USDT': {
#         'liquidity': 2062772,
#         'LRNA': 2062772
#     },
#     'DOT': {
#         # 'liquidity': 389000,
#         'liquidity': 350000,
#         'LRNA': 1456248
#     },
#     'HDX': {
#         'liquidity': 108000000,
#         'LRNA': 494896
#     }
# }

pprint(tokens)


{'asset_fee': 2500, 'protocol_fee': 500, 'timestamp': 3675543}
{'asset_fee': 2500, 'protocol_fee': 500, 'timestamp': 3675894}
{'asset_fee': 3128, 'protocol_fee': 500, 'timestamp': 3675902}
{'asset_fee': 2500, 'protocol_fee': 1000, 'timestamp': 3675899}
{'asset_fee': 2500, 'protocol_fee': 500, 'timestamp': 3570908}
{'asset_fee': 2500, 'protocol_fee': 500, 'timestamp': 3675819}
{'asset_fee': 2500, 'protocol_fee': 500, 'timestamp': 3675876}
{'asset_fee': 2500, 'protocol_fee': 632, 'timestamp': 3675547}
{'asset_fee': 2500, 'protocol_fee': 500, 'timestamp': 3675720}
{'asset_fee': 2500, 'protocol_fee': 500, 'timestamp': 3675603}
{'asset_fee': 2500, 'protocol_fee': 500, 'timestamp': 3661023}
{'ASTR': {'asset_fee': 0.0025, 'protocol_fee': 0.0005},
 'BNC': {'asset_fee': 0.0025, 'protocol_fee': 0.0005},
 'CFG': {'asset_fee': 0.0025, 'protocol_fee': 0.0006320000000000001},
 'DOT': {'asset_fee': 0.0025, 'protocol_fee': 0.0005},
 'GLMR': {'asset_fee': 0.0025, 'protocol_fee': 0.0005},
 'HDX': {'asse

In [60]:

# dot_usdt_order_book = {
#     'bids': [{'price': 3.60, 'amount': 200},
#              {'price': 3.59, 'amount': 100},
#              {'price': 3.50, 'amount': 100},
#              {'price': 3.40, 'amount': 2000}],
#     'asks': [{'price': 3.70, 'amount': 100},
#              {'price': 3.74, 'amount': 5000},
#              {'price': 3.80, 'amount': 200},
#              {'price': 3.90, 'amount': 2000}]
# }
# 
# hdx_usdt_order_book = {
#     'bids': [{'price': 0.03, 'amount': 2000},
#              {'price': 0.025, 'amount': 2000},
#              {'price': 0.02, 'amount': 2000},
#              {'price': 0.015, 'amount': 2000}],
#     'asks': [{'price': 0.04, 'amount': 2000},
#              {'price': 0.05, 'amount': 2000},
#              {'price': 0.06, 'amount': 2000},
#              {'price': 0.07, 'amount': 2000}]
# }

# hdx_dot_order_book = {
#     'bids': [{'price': 0.005, 'amount': 2000},
#              {'price': 0.004, 'amount': 2000}],
#     'asks': [{'price': 0.0052, 'amount': 2000},
#              {'price': 0.0055, 'amount': 2000}]
# }

ob_objs = []
for tkn_pair in order_books:
    ob = parse_kraken_orderbook(order_books[tkn_pair])
    ob_obj = OrderBook(
        bids=[[bid['price'], bid['amount']] for bid in ob['bids']],
        asks=[[ask['price'], ask['amount']] for ask in ob['asks']]
    )
    ob_objs.append(ob_obj)

order_book = {op_tkn_pairs[i]: ob_objs[i] for i in range(len(op_tkn_pairs))}
# order_book = {
#     ('DOT','USDT'): dot_usdt_order_book_obj,
#     ('HDX','USDT'): hdx_usdt_order_book_obj,
#     # ('HDX','DOT'): hdx_dot_order_book
# }



lrna_fee = 0.0005
asset_fee = 0.0025
cex_fee = 0.0016
buffer = 0.0002
# 
# lrna_fee = 0.0001
# asset_fee = 0.0001
# cex_fee = 0.0001
# buffer = 0.0

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}


cex = CentralizedMarket(
    order_book=order_book,
    asset_list=asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()


In [61]:
all_swaps = get_arb_swaps(op_state, cex, buffer=buffer)
for tkn_pair in all_swaps:
    if all_swaps[tkn_pair] != []:
        print(tkn_pair)
        print(all_swaps[tkn_pair])

('DOT', 'USDT')
[{'dex': {'trade': 'sell', 'price': 4.1328, 'amount': 463.4395193510055, 'min_buy': 1922.626831104827}, 'cex': {'trade': 'buy', 'price': 4.13321328, 'amount': 463.4395193510055}}]
('WETH', 'USDT')
[{'dex': {'trade': 'buy', 'price': 1692.8, 'amount': 4.922901124954224, 'max_sell': 8159.427602597749}, 'cex': {'trade': 'sell', 'price': 1692.6307199999999, 'amount': 4.922901124954224}}]
('DOT', 'WETH')
[{'dex': {'trade': 'sell', 'price': 0.002443, 'amount': 540.7639668216707, 'min_buy': 1.3308470836758615}, 'cex': {'trade': 'buy', 'price': 0.0024432443, 'amount': 540.7639668216707}}]
('WBTC', 'USDT')
[{'dex': {'trade': 'buy', 'price': 31063.0, 'amount': 0.004045160936336519, 'max_sell': 120.40939588814973}, 'cex': {'trade': 'sell', 'price': 31059.8937, 'amount': 0.004045160936336519}}]
('DOT', 'WBTC')
[{'dex': {'trade': 'sell', 'price': 0.00013311, 'amount': 3.3468284378051756, 'min_buy': 0.00044830130338668824}, 'cex': {'trade': 'buy', 'price': 0.000133123311, 'amount': 3.

In [62]:
state = init_state.copy()
init_agent = Agent(holdings={asset: 1000000000 for asset in asset_list}, unique_id='bot')
agent = init_agent.copy()
execute_arb(state, cex, agent, all_swaps)

profit = calculate_profit(init_agent, agent)

print(profit)
            
# for tkn_pair in all_swaps:
#     print(tkn_pair)
#     op_spot = OmnipoolState.price(state, tkn_pair[0], tkn_pair[1])
#     buy_spot = op_spot / ((1 - lrna_fee)*(1 - asset_fee))
#     sell_spot = op_spot * (1 - lrna_fee) * (1 - asset_fee)
#     print((sell_spot, op_spot, buy_spot))
    


{'USDT': 171.04500579833984, 'DOT': 0.0, 'HDX': 0, 'WETH': 0.007776737213134766, 'WBTC': 2.1457672119140625e-06, 'IBTC': 0, 'ASTR': 0, 'CFG': 0, 'BNC': 0, 'GLMR': 0, 'ZTG': 0}
